In [2]:
import time
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from matplotlib import cm
from collections import OrderedDict
import matplotlib as mpl
import matplotlib.colors as colors
import re

import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting
import seaborn as sns

pd.set_option('display.max_rows', 100)

bokeh.io.output_notebook

import scipy.stats as sps

In [6]:
start = time.time()

#------------------------------Using Raw Census PLine Output---------------------------------

def extract_PLines(census_file):
    """
        Input: Census filled file (extracting intensities from P-Line)
        Output: New dataframe containing data from census PLine
    """
    with open(census_file, 'r') as f:
        file1 = f.readlines()
    with open('dump1.txt', 'w') as g:    
        for line1 in file1:
            line2 = line1.split('\t')
            if line2[0] == 'PLINE':
                g.write(line1)
            elif line2[0] == 'P':
                g.write(line1)
    temp_df1 = pd.read_csv('dump1.txt', sep='\t', na_values='-').fillna(0)
    return temp_df1

def extract_Norm_Raw_PLine_Intensities(temp_df_NR1):
    """
        Input: dataframe derived from 'extract_PLines' 
        Output: new dataframe containing normalized intensities for each protein/sample
    """
    #Import Select Columns to new dataframe then add 1 to every intensity value (to ensure no 0 values)
    temp_df_NR2 = temp_df_NR1[['ACCESSION', 'DESCRIPTION', 'NORM_INTENSITY_1', 'NORM_INTENSITY_2', 'NORM_INTENSITY_3', 'NORM_INTENSITY_4', 'NORM_INTENSITY_5', 'NORM_INTENSITY_6', 'NORM_INTENSITY_7', 'NORM_INTENSITY_8', 'NORM_INTENSITY_9', 'NORM_INTENSITY_10', 'NORM_INTENSITY_11', 'NORM_INTENSITY_12', 'NORM_INTENSITY_13', 'NORM_INTENSITY_14', 'NORM_INTENSITY_15', 'NORM_INTENSITY_16', 'NORM_INTENSITY_17']].copy()
    temp_df_NR2[['NORM_INTENSITY_1', 'NORM_INTENSITY_2', 'NORM_INTENSITY_3', 'NORM_INTENSITY_4', 'NORM_INTENSITY_5', 'NORM_INTENSITY_6', 'NORM_INTENSITY_7', 'NORM_INTENSITY_8', 'NORM_INTENSITY_9', 'NORM_INTENSITY_10', 'NORM_INTENSITY_11', 'NORM_INTENSITY_12', 'NORM_INTENSITY_13', 'NORM_INTENSITY_14', 'NORM_INTENSITY_15', 'NORM_INTENSITY_16', 'NORM_INTENSITY_17']] += 1
    return temp_df_NR2

def extract_Raw_PLine_Intensities(temp_df_R1):
    """
        Input: dataframe derived from 'extract_PLines'
        Output: new dataframe containing non-normalized intensities for each protein/sample
    """
    #Import Select Columns to new dataframe then add 1 to every intensity value (to ensure no 0 values)
    temp_df_R2 = temp_df_R1[['ACCESSION', 'DESCRIPTION', 'INTENSITY_1', 'INTENSITY_2', 'INTENSITY_3', 'INTENSITY_4', 'INTENSITY_5', 'INTENSITY_6', 'INTENSITY_7', 'INTENSITY_8', 'INTENSITY_9', 'INTENSITY_10', 'INTENSITY_11', 'INTENSITY_12', 'INTENSITY_13', 'INTENSITY_14', 'INTENSITY_15', 'INTENSITY_16', 'INTENSITY_17']].copy()
    temp_df_R2[['INTENSITY_1', 'INTENSITY_2', 'INTENSITY_3', 'INTENSITY_4', 'INTENSITY_5', 'INTENSITY_6', 'INTENSITY_7', 'INTENSITY_8', 'INTENSITY_9', 'INTENSITY_10', 'INTENSITY_11', 'INTENSITY_12', 'INTENSITY_13', 'INTENSITY_14', 'INTENSITY_15', 'INTENSITY_16', 'INTENSITY_17']] += 1
    return temp_df_R2

def normalize_Raw_PLine_Intensities(temp_df_R3):
    """
        Input: non-normalized intensities dataframe from 'extract_Raw_PLine_Intensities' function
        Output: return same dataframe with normalized intensity values; each value in a column is divided by its specific column sum
    """
    temp_allsums1 = temp_df_R3.sum(axis=0, skipna = True)
    temp_df_R3['INTENSITY_1'] /= temp_allsums1['INTENSITY_1']
    temp_df_R3['INTENSITY_2'] /= temp_allsums1['INTENSITY_2']
    temp_df_R3['INTENSITY_3'] /= temp_allsums1['INTENSITY_3']
    temp_df_R3['INTENSITY_4'] /= temp_allsums1['INTENSITY_4']
    temp_df_R3['INTENSITY_5'] /= temp_allsums1['INTENSITY_5']
    temp_df_R3['INTENSITY_6'] /= temp_allsums1['INTENSITY_6']
    temp_df_R3['INTENSITY_7'] /= temp_allsums1['INTENSITY_7']
    temp_df_R3['INTENSITY_8'] /= temp_allsums1['INTENSITY_8']
    temp_df_R3['INTENSITY_9'] /= temp_allsums1['INTENSITY_9']
    temp_df_R3['INTENSITY_10'] /= temp_allsums1['INTENSITY_10']
    temp_df_R3['INTENSITY_11'] /= temp_allsums1['INTENSITY_11']
    temp_df_R3['INTENSITY_12'] /= temp_allsums1['INTENSITY_12']
    temp_df_R3['INTENSITY_13'] /= temp_allsums1['INTENSITY_13']
    temp_df_R3['INTENSITY_14'] /= temp_allsums1['INTENSITY_14']
    temp_df_R3['INTENSITY_15'] /= temp_allsums1['INTENSITY_15']
    temp_df_R3['INTENSITY_16'] /= temp_allsums1['INTENSITY_16']
    temp_df_R3['INTENSITY_17'] /= temp_allsums1['INTENSITY_17']
    return temp_df_R3

#-------------------------------------------------------------------------------------------

#----------------------------Clustering Proteins - Census SLine-----------------------------


def extract_SLine_from_Census(census_file):
    """
        Input: Census 'filled' file
        Output: text dump and returns a peptide dataframe
    """
    with open(census_file, 'r') as f:
        file1 = f.readlines()
    with open('dump1.txt', 'w') as g:
        for line1 in file1:
            if (line1[0] == 'SLINE') or (line1[0] == 'S'):
                g.write(line1)
    df1 = pd.read_csv('dump1.txt', sep='\t', header=0).fillna('0')
    return df1

def select_Census_Columns_Peptides(temp_df1):
    """
        Input: raw census peptide (SLine) dataframe [Reliant on 'extract_SLine_from_Census']
        Output: new cleaned up dataframe containing peptides (no duplicates) and corresponding intensities for each run
    """
    #Note that columns below represent Peptide sequence and Intensity Columns -- CHANGE AS NECESSARY
    temp_df2 = temp_df1[['SEQUENCE_1','SEQUENCE_2','SEQUENCE_3','SEQUENCE_4','SEQUENCE_5','SEQUENCE_6','SEQUENCE_7','SEQUENCE_8','SEQUENCE_9','SEQUENCE_10','SEQUENCE_11','SEQUENCE_12','SEQUENCE_13','SEQUENCE_14','SEQUENCE_15','SEQUENCE_16','SEQUENCE_17','INTENSITY_1','INTENSITY_2','INTENSITY_3','INTENSITY_4','INTENSITY_5','INTENSITY_6','INTENSITY_7','INTENSITY_8','INTENSITY_9','INTENSITY_10','INTENSITY_11','INTENSITY_12','INTENSITY_13','INTENSITY_14','INTENSITY_15','INTENSITY_16','INTENSITY_17']].copy()
    
    #Find sequence from dataframe SEQUENCE_# values
    temp_SEQUENCE = []
    for row1 in temp_df2.itertuples():
        temp_list1 = []
        temp_list1.append(row1.SEQUENCE_1)
        temp_list1.append(row1.SEQUENCE_2)
        temp_list1.append(row1.SEQUENCE_3)
        temp_list1.append(row1.SEQUENCE_4)
        temp_list1.append(row1.SEQUENCE_5)
        temp_list1.append(row1.SEQUENCE_6)
        temp_list1.append(row1.SEQUENCE_7)
        temp_list1.append(row1.SEQUENCE_8)
        temp_list1.append(row1.SEQUENCE_9)
        temp_list1.append(row1.SEQUENCE_10)
        temp_list1.append(row1.SEQUENCE_11)
        temp_list1.append(row1.SEQUENCE_12)
        temp_list1.append(row1.SEQUENCE_13)
        temp_list1.append(row1.SEQUENCE_14)
        temp_list1.append(row1.SEQUENCE_15)  
        temp_list1.append(row1.SEQUENCE_16)
        temp_list1.append(row1.SEQUENCE_17)
        item2 = 0
        for item1 in temp_list1:
            if item1 != '0':
                item2 = item1
                break
        temp_SEQUENCE.append(item2)
    temp_df2['SEQUENCE'] = pd.Series(temp_SEQUENCE, index=temp_df2.index)
    temp_df2 = temp_df2[['SEQUENCE','INTENSITY_1','INTENSITY_2','INTENSITY_3','INTENSITY_4','INTENSITY_5','INTENSITY_6','INTENSITY_7','INTENSITY_8','INTENSITY_9','INTENSITY_10','INTENSITY_11','INTENSITY_12','INTENSITY_13','INTENSITY_14','INTENSITY_15','INTENSITY_16','INTENSITY_17']].copy()
    temp_df2['INTENSITY_1'] = pd.to_numeric(temp_df2['INTENSITY_1'], errors='coerce')
    temp_df2['INTENSITY_2'] = pd.to_numeric(temp_df2['INTENSITY_2'], errors='coerce')
    temp_df2['INTENSITY_3'] = pd.to_numeric(temp_df2['INTENSITY_3'], errors='coerce')
    temp_df2['INTENSITY_4'] = pd.to_numeric(temp_df2['INTENSITY_4'], errors='coerce')
    temp_df2['INTENSITY_5'] = pd.to_numeric(temp_df2['INTENSITY_5'], errors='coerce')
    temp_df2['INTENSITY_6'] = pd.to_numeric(temp_df2['INTENSITY_6'], errors='coerce')
    temp_df2['INTENSITY_7'] = pd.to_numeric(temp_df2['INTENSITY_7'], errors='coerce')
    temp_df2['INTENSITY_8'] = pd.to_numeric(temp_df2['INTENSITY_8'], errors='coerce')
    temp_df2['INTENSITY_9'] = pd.to_numeric(temp_df2['INTENSITY_9'], errors='coerce')
    temp_df2['INTENSITY_10'] = pd.to_numeric(temp_df2['INTENSITY_10'], errors='coerce')
    temp_df2['INTENSITY_11'] = pd.to_numeric(temp_df2['INTENSITY_11'], errors='coerce')
    temp_df2['INTENSITY_12'] = pd.to_numeric(temp_df2['INTENSITY_12'], errors='coerce')
    temp_df2['INTENSITY_13'] = pd.to_numeric(temp_df2['INTENSITY_13'], errors='coerce')
    temp_df2['INTENSITY_14'] = pd.to_numeric(temp_df2['INTENSITY_14'], errors='coerce')
    temp_df2['INTENSITY_15'] = pd.to_numeric(temp_df2['INTENSITY_15'], errors='coerce')
    temp_df2['INTENSITY_16'] = pd.to_numeric(temp_df2['INTENSITY_16'], errors='coerce')
    temp_df2['INTENSITY_17'] = pd.to_numeric(temp_df2['INTENSITY_17'], errors='coerce')
    temp_df2 = temp_df2.fillna(0)
    
    #Remove rows containing duplicate information (i.e. intensities); Duplicate peptides with different intensities are kept
    temp_df3 = temp_df2.drop_duplicates(keep='first', inplace=False)
    
    #Reformatting Peptides to remove C- and N-terminal cleavage sites and diff-mod sites
    temp_col1 = temp_df3['SEQUENCE']
    temp_col2 = []
    for temp_position1,temp_item1 in enumerate(temp_col1):
        temp_col2.append(temp_item1[2:-2].replace('(15.994915)', ''))
    temp_df3 = temp_df3.reset_index()
    del temp_df3['index']
    temp_df4 = temp_df3.copy()
    temp_df4['PEPTIDE'] = pd.Series(temp_col2, index=temp_df4.index)
    
    #Create new cleaned up dataframe
    temp_df5 = temp_df4[['PEPTIDE', 'INTENSITY_1', 'INTENSITY_2', 'INTENSITY_3', 'INTENSITY_4', 'INTENSITY_5', 'INTENSITY_6', 'INTENSITY_7', 'INTENSITY_8', 'INTENSITY_9', 'INTENSITY_10', 'INTENSITY_11', 'INTENSITY_12', 'INTENSITY_13', 'INTENSITY_14', 'INTENSITY_15', 'INTENSITY_16', 'INTENSITY_17']]
    temp_df6 = temp_df5.drop_duplicates(keep='first', inplace=False)
    temp_df6 = temp_df6.reset_index()
    del temp_df6['index']
    
    #Sort new cleaned up dataframe and sum intensities columns for identical peptides
    temp_df6.sort_values('PEPTIDE')
    temp_df7 = temp_df6.groupby(['PEPTIDE']).sum()
    return temp_df7

def extract_PLine_from_Census(census_file):
    """
        Input: census "filled" file
        Output: text dump and returns a dictionary connecting peptides (key) and protein list (value) redundant proteins included
    """
    #open census file
    with open(census_file, 'r') as f:
        file1 = f.readlines()
    #generate new file containing protein PLines
    with open('dump2.txt', 'w') as g:
        temp_pep_list1 = []
        for line1 in file1:
            line2 = line1.split('\t')
            if (line2[0] == 'P'):
                temp_pep_list2 = list(set(temp_pep_list1))
                for item1 in temp_pep_list2:
                    g.write('\t' + item1 + '\n')
                temp_pep_list1 = []
                g.write(line2[1] + '\t' + line2[2] + '\n')
            elif (line2[0] == 'S'):
                item02 = ''
                for item01 in line2:
                    if '-.' in item01:
                        item02 = item01
                        break
                temp_pep_list1.append(item02[2:-2].replace('(15.994915)', ''))
        temp_pep_list2 = list(set(temp_pep_list1))
        for item1 in temp_pep_list2:
            g.write('\t' + item1 + '\n')
            temp_pep_list1 = []
    
    #open newly created protein PLine file
    with open('dump2.txt', 'r') as f:
        file2 = f.readlines()
    
    #create peptide(key)-protein(value) dictionary; proteins are assembled in list
    Pep_to_Prot_Dict1 = {}
    temp_prot1 = ""
    for line1 in file2:
        line2 = line1.split('\t')
        if line2[0] != '':
            temp_prot1 = line2[0]
        else:
            if line2[1].replace('\n', '') in Pep_to_Prot_Dict1.keys():
                Pep_to_Prot_Dict1[line2[1].replace('\n', '')].append(temp_prot1)
            else:
                Pep_to_Prot_Dict1[line2[1].replace('\n', '')] = []
                Pep_to_Prot_Dict1[line2[1].replace('\n', '')].append(temp_prot1)
    return Pep_to_Prot_Dict1

def map_Clusters_to_Proteins(cluster_file):
    """
        Input: CDHIT cluster file
        Output: dictionary connecting Protein (keys) to Cluster number (values)
    """
    #create dictionary connecting protein (keys) and cluster number (values)
    with open(cluster_file, 'r') as f:
        file1 = f.readlines() 
    temp_cluster_num = ""
    Protein_to_Cluster_Dict1 = {}
    for line1 in file1:
        line2 = line1.split('\t')        
        if '>' in line2[0]:
            temp_cluster_num = line2[0].replace('>', '').replace('\n', '').replace('Cluster ', '')
        else:
            line3 = line2[1].split(' ')
            line4 = line3[1].replace('>', '').replace('...', '')
            if '|' in line4:
                line4b = line4.split('|')
                line4 = line4b[1]
            if line4[0:7] != 'Reverse':
                Protein_to_Cluster_Dict1[line4] = temp_cluster_num
    return Protein_to_Cluster_Dict1

def map_Clusternum_to_Peptides(prot_to_clust_dict2, pep_to_prot_dict2):
    """
        Input: 2 dictionaries: protein-cluster and peptide-protein
        Output: dictionary linking peptides(key) to cluster number (value)
    """
    pep_to_cluster_dict1 = {}
    for item1 in pep_to_prot_dict2.keys():
        temp_protlist1 = pep_to_prot_dict2[item1]
        temp_cluster_list1 = []
        for item2 in temp_protlist1:
            try:
                temp_cluster_list1.append(prot_to_clust_dict2[item2])
            except:
                pass
        pep_to_cluster_dict1[item1] = list(set(temp_cluster_list1))
    return pep_to_cluster_dict1

def clusternum_to_PepDataframe(pep_to_clust_dict, uniquepep_dataframe):
    """
        Input: Peptide-to-Cluster dictionary generated from 'map_Clusternum_to_Peptides' function and cleaned-up Peptide-Intensity dataframe from 'select_Census_Columns_Peptides' function
        Output: New peptide-intensity dataframe with integrated cluster number column; Note that peptides with no cluster belong to contaminant or reverse proteins
    """
    temp_df1 = uniquepep_dataframe.reset_index()
    temp_list1 = list(temp_df1['PEPTIDE'])
    temp_clusterlist1 = []
    for item1 in temp_list1:
        temp_clusterlist1.append(pep_to_clust_dict[item1])
    temp_df1['CLUSTER'] = pd.Series(temp_clusterlist1, index=temp_df1.index)
    temp_df2 = temp_df1[['PEPTIDE', 'CLUSTER', 'INTENSITY_1', 'INTENSITY_2', 'INTENSITY_3', 'INTENSITY_4', 'INTENSITY_5', 'INTENSITY_6', 'INTENSITY_7', 'INTENSITY_8', 'INTENSITY_9', 'INTENSITY_10', 'INTENSITY_11', 'INTENSITY_12', 'INTENSITY_13', 'INTENSITY_14', 'INTENSITY_15', 'INTENSITY_16', 'INTENSITY_17']]
    return temp_df2

def extract_Intensity_Master(census_file1, cluster_file1):
    """
        Input: census 'filled' file and CDHIT cluster file
        Output: Dataframe with Peptide, Cluster, and Intensity Columns
    """
    temp_df1A = extract_SLine_from_Census(census_file1)
    temp_df2A = select_Census_Columns_Peptides(temp_df1A)
    temp_dict1A = extract_PLine_from_Census(census_file1)
    temp_dict2A = map_Clusters_to_Peptides(cluster_file1)
    temp_dict3A = map_Clusternum_to_Peptides(temp_dict2A, temp_dict1A)
    temp_df3A = clusternum_to_PepDataframe(temp_dict3A, temp_df2A)
    return temp_df3A

def create_Cluster_Intensity_Table(complete_dataframe):
    """
        Input: complete master dataframe from 'extract_Intensity_Master'
        Output: new dataframe with intensities summed by cluster
    """
    #Collect only peptides that map to 1 cluster
    temp_df1A = complete_dataframe.loc[complete_dataframe['CLUSTER'].str.len() == 1]
    #Create new column (ClusterID) then sum intensities belonging to the same cluster
    temp_list1 = temp_df1A['CLUSTER']
    temp_list2 = []
    for item1 in temp_list1:
        temp_list2.append(item1[0])
    temp_df1A['ClusterID'] = pd.Series(temp_list2, index=temp_df1A.index)
    temp_df1A = temp_df1A[['PEPTIDE', 'CLUSTER', 'ClusterID', 'INTENSITY_1', 'INTENSITY_2', 'INTENSITY_3', 'INTENSITY_4', 'INTENSITY_5', 'INTENSITY_6', 'INTENSITY_7', 'INTENSITY_8', 'INTENSITY_9', 'INTENSITY_10', 'INTENSITY_11', 'INTENSITY_12', 'INTENSITY_13', 'INTENSITY_14', 'INTENSITY_15', 'INTENSITY_16', 'INTENSITY_17']]
    cluster_df1 = temp_df1A.groupby(['ClusterID']).sum().copy()    
    #Add 1 to each value to ensure no 0 values (necessary for Log transform and subsequent hypothesis testing)
    cluster_df1[['INTENSITY_1', 'INTENSITY_2', 'INTENSITY_3', 'INTENSITY_4', 'INTENSITY_5', 'INTENSITY_6', 'INTENSITY_7', 'INTENSITY_8', 'INTENSITY_9', 'INTENSITY_10', 'INTENSITY_11', 'INTENSITY_12', 'INTENSITY_13', 'INTENSITY_14', 'INTENSITY_15', 'INTENSITY_16', 'INTENSITY_17']] += 0
    return cluster_df1

def normalize_Cluster_Intensity_Table(complete_dataframe_2):
    """
        Input: Cluster-Intensity dataframe
        Output: New Cluster-Intensity dataframe with each entry normalized by dividing by respective column sum (note that intensities from peptides mapping to multiple clusters tossed)
    """
    temp_allsums1 = complete_dataframe_2.sum(axis=0, skipna = True)
    complete_dataframe_2['INTENSITY_1'] /= temp_allsums1['INTENSITY_1']
    complete_dataframe_2['INTENSITY_2'] /= temp_allsums1['INTENSITY_2']
    complete_dataframe_2['INTENSITY_3'] /= temp_allsums1['INTENSITY_3']
    complete_dataframe_2['INTENSITY_4'] /= temp_allsums1['INTENSITY_4']
    complete_dataframe_2['INTENSITY_5'] /= temp_allsums1['INTENSITY_5']
    complete_dataframe_2['INTENSITY_6'] /= temp_allsums1['INTENSITY_6']
    complete_dataframe_2['INTENSITY_7'] /= temp_allsums1['INTENSITY_7']
    complete_dataframe_2['INTENSITY_8'] /= temp_allsums1['INTENSITY_8']
    complete_dataframe_2['INTENSITY_9'] /= temp_allsums1['INTENSITY_9']
    complete_dataframe_2['INTENSITY_10'] /= temp_allsums1['INTENSITY_10']
    complete_dataframe_2['INTENSITY_11'] /= temp_allsums1['INTENSITY_11']
    complete_dataframe_2['INTENSITY_12'] /= temp_allsums1['INTENSITY_12']
    complete_dataframe_2['INTENSITY_13'] /= temp_allsums1['INTENSITY_13']
    complete_dataframe_2['INTENSITY_14'] /= temp_allsums1['INTENSITY_14']
    complete_dataframe_2['INTENSITY_15'] /= temp_allsums1['INTENSITY_15']
    complete_dataframe_2['INTENSITY_16'] /= temp_allsums1['INTENSITY_16']
    complete_dataframe_2['INTENSITY_17'] /= temp_allsums1['INTENSITY_17']
    return complete_dataframe_2


#-------------------------------------------------------------------------------------------

#----------------------------Formatting and Annotating R-output-----------------------------

def annotate_R_abridged(cdhit_file, abridged_R_file):
    """
        Input: CDHIT cluster file and abridged R outputfile containing p-adj values
        Output: newly annotated dataframe
    """
    with open(cdhit_file, 'r') as f:
        file1 = f.readlines()
    cluster_representative_dict1 = {}
    for line1 in file1:
        line2 = line1.split('\t')
        if line1[0] == '>':
            temp_clusternum = line2[0].replace('>Cluster ', '').replace('\n', '')
        elif '... *' in line2[1]:
            line3 = line2[1].split(' ')
            line4 = line3[1].replace('>', '').replace('...', '')
            cluster_representative_dict1[temp_clusternum] = line4
            temp_clusternum = ""
    with open(abridged_R_file, 'r') as g:
        file2 = g.readlines()
    cluster_rep_list = []
    for line1 in file2:
        line2 = line1.split(',')
        if line2[1] in cluster_representative_dict1.keys():
            cluster_rep_list.append(cluster_representative_dict1[line2[1]])
    temp_dfC1 = pd.read_csv(abridged_R_file) 
    temp_dfC1['cluster_representative'] = pd.Series(cluster_rep_list, index=temp_dfC1.index)
    return temp_dfC1

#-------------------------------------------------------------------------------------------

end = time.time()
print(end-start)

0.002644062042236328


In [15]:
start = time.time()

#---------------Prepare Tables for R-analysis of Census PLine Data-------------------

#convert census_filled into file with only PLines
censusP_df1 = extract_PLines('Census/census-chat-mouse-20318_filled_modded.txt')

#extract specific columns from converted census file
Norm_PLine_df1 = extract_Norm_Raw_PLine_Intensities(censusP_df1)
Non_Norm_PLine_df1 = extract_Raw_PLine_Intensities(censusP_df1)

#normalize columns in Non_Norm extracted file
Non_Norm_PLine_df1 = normalize_Raw_PLine_Intensities(Non_Norm_PLine_df1)

#output files to csv ready for R-analysis
Norm_PLine_df1.to_csv('PLine-ChatMouse-NormIntensityTable1.txt', index=False)
Non_Norm_PLine_df1.to_csv('PLine-ChatMouse-NonNormIntensityTable1.txt', index=False)

#------------------------------------------------------------------------------------

end = time.time()
print(end-start)

1.121624231338501


In [67]:
start = time.time()

#Only need to run once for each CDHIT-# analysis (take ~1.5 hours)
#temp_df1A = extract_SLine_from_Census('Census/census-chat-microbiome-20338_modded.txt')

#--------Prepare Tables for R-analysis of Census SLine Data (Clustering)-------------

#CHAT-Microbiome, CDHIT 65 (Run once per analysis)
temp_df2A = select_Census_Columns_Peptides(temp_df1A)
temp_dict1A = extract_PLine_from_Census('Census/census-chat-microbiome-20338_modded.txt')
temp_dict2A = map_Clusters_to_Proteins('CDHIT/cdhitout-chat-65.clstr')
temp_dict3A = map_Clusternum_to_Peptides(temp_dict2A, temp_dict1A)
temp_chatMB_65 = clusternum_to_PepDataframe(temp_dict3A, temp_df2A)
chatMB_65 = create_Cluster_Intensity_Table(temp_chatMB_65)
#chatMB_65 = normalize_Cluster_Intensity_Table(chatMB_65)

#CHAT-Microbiome, CDHIT 75 (Run once per analysis)
temp_df2A = select_Census_Columns_Peptides(temp_df1A)
temp_dict1A = extract_PLine_from_Census('Census/census-chat-microbiome-20338_modded.txt')
temp_dict2A = map_Clusters_to_Proteins('CDHIT/cdhitout-chat-75.clstr')
temp_dict3A = map_Clusternum_to_Peptides(temp_dict2A, temp_dict1A)
temp_chatMB_75 = clusternum_to_PepDataframe(temp_dict3A, temp_df2A)
chatMB_75 = create_Cluster_Intensity_Table(temp_chatMB_75)
#chatMB_75 = normalize_Cluster_Intensity_Table(chatMB_75)

#CHAT-Microbiome, CDHIT 85 (Run once per analysis)
temp_df2A = select_Census_Columns_Peptides(temp_df1A)
temp_dict1A = extract_PLine_from_Census('Census/census-chat-microbiome-20338_modded.txt')
temp_dict2A = map_Clusters_to_Proteins('CDHIT/cdhitout-chat-85.clstr')
temp_dict3A = map_Clusternum_to_Peptides(temp_dict2A, temp_dict1A)
temp_chatMB_85 = clusternum_to_PepDataframe(temp_dict3A, temp_df2A)
chatMB_85 = create_Cluster_Intensity_Table(temp_chatMB_85)
#chatMB_85 = normalize_Cluster_Intensity_Table(chatMB_85)

#CHAT-Microbiome, CDHIT 95 (Run once per analysis)
temp_df2A = select_Census_Columns_Peptides(temp_df1A)
temp_dict1A = extract_PLine_from_Census('Census/census-chat-microbiome-20338_modded.txt')
temp_dict2A = map_Clusters_to_Proteins('CDHIT/cdhitout-chat-95.clstr')
temp_dict3A = map_Clusternum_to_Peptides(temp_dict2A, temp_dict1A)
temp_chatMB_95 = clusternum_to_PepDataframe(temp_dict3A, temp_df2A)
chatMB_95 = create_Cluster_Intensity_Table(temp_chatMB_95)
#chatMB_95 = normalize_Cluster_Intensity_Table(chatMB_95)

#Output to txt/csv file
chatMB_65.to_csv('SLine-ChatMB-NormIntensityTable-65.txt')
chatMB_75.to_csv('SLine-ChatMB-NormIntensityTable-75.txt')
chatMB_85.to_csv('SLine-ChatMB-NormIntensityTable-85.txt')
chatMB_95.to_csv('SLine-ChatMB-NormIntensityTable-95.txt')


#------------------------------------------------------------------------------------

end = time.time()
print(end-start)

/Users/PTB/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


162.32936120033264


In [20]:
chatMB_65[chatMB_65.eq(0).any(1)]

,INTENSITY_1,INTENSITY_2,INTENSITY_3,INTENSITY_4,INTENSITY_5,INTENSITY_6,INTENSITY_7,INTENSITY_8,INTENSITY_9,INTENSITY_10,INTENSITY_11,INTENSITY_12,INTENSITY_13,INTENSITY_14,INTENSITY_15,INTENSITY_16,INTENSITY_17
ClusterID,,,,,,,,,,,,,,,,,


In [3]:
start = time.time()

#---------------Add Protein Labels to R-analysis of Census SLine Data----------------

chatmouse_65_t_R1 = annotate_R_abridged('CDHIT/cdhitout-chat-65.clstr', 'Census_analysis/SLine-ChatMB-Census-Log2Transform-65.csv')
chatmouse_65_t_R1.drop(columns='Unnamed: 0', inplace=True)
chatmouse_65_t_R1.to_csv('SLine-ChatMB-Census-Log2Transform-65-annotated.txt', index=False)

chatmouse_75_t_R1 = annotate_R_abridged('CDHIT/cdhitout-chat-75.clstr', 'Census_analysis/SLine-ChatMB-Census-Log2Transform-75.csv')
chatmouse_75_t_R1.drop(columns='Unnamed: 0', inplace=True)
chatmouse_75_t_R1.to_csv('SLine-ChatMB-Census-Log2Transform-75-annotated.txt', index=False)

chatmouse_85_t_R1 = annotate_R_abridged('CDHIT/cdhitout-chat-85.clstr', 'Census_analysis/SLine-ChatMB-Census-Log2Transform-85.csv')
chatmouse_85_t_R1.drop(columns='Unnamed: 0', inplace=True)
chatmouse_85_t_R1.to_csv('SLine-ChatMB-Census-Log2Transform-85-annotated.txt', index=False)

chatmouse_95_t_R1 = annotate_R_abridged('CDHIT/cdhitout-chat-95.clstr', 'Census_analysis/SLine-ChatMB-Census-Log2Transform-95.csv')
chatmouse_95_t_R1.drop(columns='Unnamed: 0', inplace=True)
chatmouse_95_t_R1.to_csv('SLine-ChatMB-Census-Log2Transform-95-annotated.txt', index=False)


#----------

chatmouse_65_t_R1 = annotate_R_abridged('CDHIT/cdhitout-chat-65.clstr', 'Census_analysis/SLine-ChatMB-Census-cuberootTransform-65.csv')
chatmouse_65_t_R1.drop(columns='Unnamed: 0', inplace=True)
chatmouse_65_t_R1.to_csv('SLine-ChatMB-Census-cuberootTransform-65-annotated.txt', index=False)

chatmouse_75_t_R1 = annotate_R_abridged('CDHIT/cdhitout-chat-75.clstr', 'Census_analysis/SLine-ChatMB-Census-cuberootTransform-75.csv')
chatmouse_75_t_R1.drop(columns='Unnamed: 0', inplace=True)
chatmouse_75_t_R1.to_csv('SLine-ChatMB-Census-cuberootTransform-75-annotated.txt', index=False)

chatmouse_85_t_R1 = annotate_R_abridged('CDHIT/cdhitout-chat-85.clstr', 'Census_analysis/SLine-ChatMB-Census-cuberootTransform-85.csv')
chatmouse_85_t_R1.drop(columns='Unnamed: 0', inplace=True)
chatmouse_85_t_R1.to_csv('SLine-ChatMB-Census-cuberootTransform-85-annotated.txt', index=False)

chatmouse_95_t_R1 = annotate_R_abridged('CDHIT/cdhitout-chat-95.clstr', 'Census_analysis/SLine-ChatMB-Census-cuberootTransform-95.csv')
chatmouse_95_t_R1.drop(columns='Unnamed: 0', inplace=True)
chatmouse_95_t_R1.to_csv('SLine-ChatMB-Census-cuberootTransform-95-annotated.txt', index=False)


#------------------------------------------------------------------------------------

end = time.time()
print(end-start)

2.7230849266052246


In [239]:
temp_chatMB_65.loc[temp_chatMB_65['CLUSTER'].str.len() == 0]

,PEPTIDE,CLUSTER,INTENSITY_1,INTENSITY_2,INTENSITY_3,INTENSITY_4,INTENSITY_5,INTENSITY_6,INTENSITY_7,INTENSITY_8,INTENSITY_9,INTENSITY_10,INTENSITY_11,INTENSITY_12,INTENSITY_13,INTENSITY_14,INTENSITY_15,INTENSITY_16,INTENSITY_17


In [21]:
chatMB_65_mod = chatMB_65[(chatMB_65 != 0).all(1)]

In [16]:
chatMB_65_mod

,INTENSITY_1,INTENSITY_2,INTENSITY_3,INTENSITY_4,INTENSITY_5,INTENSITY_6,INTENSITY_7,INTENSITY_8,INTENSITY_9,INTENSITY_10,INTENSITY_11,INTENSITY_12,INTENSITY_13,INTENSITY_14,INTENSITY_15,INTENSITY_16,INTENSITY_17
ClusterID,,,,,,,,,,,,,,,,,
105,36271295.0,41177119.0,48722983.0,3.713498e+08,18707019.0,84630621.0,1.246717e+08,108314651.0,2.967816e+08,134661393.0,49488823.0,7.482563e+08,1.213554e+09,1.855307e+08,3.562673e+08,5.241841e+08,79272822.0
1098,85014707.0,30809303.0,58384466.0,6.273672e+08,136200422.0,34470792.0,5.226969e+08,18315050.0,1.098933e+09,95320282.0,51405654.0,3.469384e+07,4.057677e+08,4.529110e+08,5.426110e+08,5.567733e+08,65742725.0
1109,223791247.0,178912079.5,291389232.0,2.010431e+08,183142036.0,250305557.0,4.609302e+08,145601246.0,3.483979e+08,136853862.0,128549721.0,2.853296e+08,2.952686e+08,1.414721e+08,1.319867e+08,1.751864e+08,277092548.0
112,25036878.0,17009530.0,33637517.0,9.215520e+07,52609333.0,46997004.0,4.153678e+07,1361326.0,7.853596e+07,21273020.0,31031195.0,2.125327e+08,1.196071e+07,7.407990e+06,4.126720e+07,8.368831e+07,36682524.0
1122,13843780.0,89296192.0,18101583.0,5.198729e+08,34505584.0,62776028.0,6.622167e+07,4544984.0,1.057189e+09,210911205.0,59838685.0,2.024458e+08,3.205426e+07,3.718354e+08,1.135164e+09,1.443453e+09,64133604.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,919705367.0,741870053.0,518418418.5,6.153664e+09,828545516.0,667848789.5,3.540808e+09,331462006.0,6.341385e+09,333242206.0,516931975.0,2.247789e+09,3.985327e+09,1.296462e+09,2.876408e+09,8.858046e+09,977764513.0
874,24001426.0,13024396.0,25232577.0,1.948847e+07,34720934.0,35374249.0,7.709741e+07,7632493.0,6.164563e+07,16064269.0,11576254.0,1.983756e+07,6.392284e+07,7.968789e+06,3.707071e+07,9.551162e+07,14148838.0
897,108443383.0,254443139.0,114025200.0,1.976698e+07,78275402.0,196051245.0,1.529614e+08,55559571.0,2.790194e+08,54356141.0,26346451.0,1.280181e+08,9.748841e+07,2.391836e+07,3.478370e+07,3.083833e+07,42456345.0


In [61]:
chatMB_65_contr = chatMB_65.iloc[:,0:9].copy()
chatMB_65_treat = chatMB_65.iloc[:,9:17].copy()

In [64]:
chatMB_65_contr = chatMB_65_contr[(chatMB_65_contr != 0).any(1)]
chatMB_65_treat = chatMB_65_treat[(chatMB_65_treat != 0).any(1)]